In [7]:
import psycopg2
import folium
import os
import numpy as np
from osgeo import gdal
import ogr
import osr
import gdalnumeric
import gdalconst
import osgeo
import osgeo.ogr
import json
import requests
import pandas as pd
import json
from branca.colormap import LinearColormap
import random
import pygeoj
import os.path
import time


#progress bar
from tqdm import trange, tqdm_notebook, tqdm
from time import sleep

# Connect to an existing database
conn = psycopg2.connect("dbname='proj_matheus' port = '5432' user='user' host='localhost' password='user'")

# Open a cursor to perform database operations
cur = conn.cursor()

# Execute a command

cur.execute("""CREATE OR REPLACE VIEW teste AS SELECT * FROM pontos""")

#cur.execute("""SELECT k FROM pontos""")
#rows = cur.fetchall()
#for row in rows: 
 #   print " ", row[0]
conn.commit()

# Close communication with the database
cur.close()
conn.close()

m = folium.Map(
    location=[-25.3171, -49.2420],
    zoom_start=13,
    width=800, height=600)


# -------- COLETA DE DADOS DE INPUT


name = raw_input('Nome da amostra (sem espaco): ')

valor = raw_input('Susceptibilidade magnetica (de 0.00 a 1.00): ')


# -------- CRIAR RASTER A PARTIR DE SHAPEFILE


#ds = gdal.Open('PG:host=localhost dbname=proj_matheus table=pontos user=user password=user')

# ds2 = gdal.Grid(
#     '/home/user/jupyter/notebook_gallery/' + name + "_result" + ".tif",
# #    '/home/user/jupyter/notebook_gallery/' + name + "_raster" + ".tif",
#     'pontos.shp',
#     format='GTiff',
#     outputBounds=[670565, 7194942, 683005, 7203143],
#     width=100,
#     height=100,
#     outputType=gdal.GDT_Float32,
#     algorithm='invdist:power=2:smoothing=10:radius1=0:radius2=0:angle=0:max_points=0:min_points=0:nodata=0',
#     zfield='K'
# )

# ds2 = gdal.Grid(
#     '/home/user/jupyter/notebook_gallery/result.tif',
# #    '/home/user/jupyter/notebook_gallery/' + name + "_raster" + ".tif",
#     'pontos.shp',
#     format='GTiff',
#     outputBounds=[670565, 7194942, 683005, 7203143],
#     width=100,
#     height=100,
#     outputType=gdal.GDT_Float32,
#     algorithm='invdist:power=2:smoothing=10:radius1=0:radius2=0:angle=0:max_points=0:min_points=0:nodata=0',
#     zfield='K'
# )


# -------- RECLASSIFICAR RASTER


driver = gdal.GetDriverByName('GTiff')
file = gdal.Open('/home/user/jupyter/notebook_gallery/result.tif')
#file = gdal.Open('/home/user/jupyter/notebook_gallery/' + name + "_raster" + ".tif")
band = file.GetRasterBand(1)
lista = band.ReadAsArray()

#print(lista)

# reclassification
for j in range(file.RasterXSize):
    for i in range(file.RasterYSize):
        if lista[i,j] <= eval(valor) - 0.02:
            lista[i,j] = 1
        elif eval(valor) - 0.02 < lista[i,j] < eval(valor) + 0.02:
            lista[i,j] = 2
        else:
            lista[i,j] = 3
#print(lista)

# create new file
file2 = driver.Create(name + "_reclass" + ".tif", file.RasterXSize , file.RasterYSize , 1)
file2.GetRasterBand(1).WriteArray(lista)

# spatial ref system
proj = file.GetProjection()
georef = file.GetGeoTransform()
file2.SetProjection(proj)
file2.SetGeoTransform(georef);
#file2.FlushCache()

# create new file
file2 = driver.Create(name + "_2_reclass" + ".tif", file.RasterXSize , file.RasterYSize , 1)
file2.GetRasterBand(1).WriteArray(lista)

# spatial ref system
proj = file.GetProjection()
georef = file.GetGeoTransform()
file2.SetProjection(proj)
file2.SetGeoTransform(georef);
#file2.FlushCache()


# -------- BARRA DE PROGRESSO
        

with tqdm_notebook(total=os.stat(name + "_reclass" + ".tif").st_size) as pbar:
    for x in name + ".tif":
        pbar.update(1)
        time.sleep(0.1)
    pbar.close()


# -------- CONVERTER RASTER PARA VETOR


file_size = os.path.getsize('/home/user/jupyter/notebook_gallery/' + name + "_reclass" + ".tif")
#file_size = os.path.getsize('/home/user/jupyter/notebook_gallery/sample_12.tif')
print(file_size)

while not file_size == 10420:
    time.sleep(1)
    
if file_size == 10420:
    #os.system('gdal_polygonize.py ' + '/home/user/jupyter/notebook_gallery/sample_12.tif' + ' -f "GeoJSON" ' + name + ".geojson");
    os.system('gdal_polygonize.py ' + '/home/user/jupyter/notebook_gallery/' + name + "_reclass" + ".tif" + ' -f "GeoJSON" ' + name + ".geojson");

else: 
    raise ValueError("%s isn't ready, calm down!" % name)


# -------- REPROJETAR O VETOR DE UTM PARA WGS


driver = ogr.GetDriverByName('GeoJSON')

# input SpatialReference
inSpatialRef = osr.SpatialReference()
inSpatialRef.ImportFromEPSG(32722)

# output SpatialReference
outSpatialRef = osr.SpatialReference()
outSpatialRef.ImportFromEPSG(4326)

# create the CoordinateTransformation
coordTrans = osr.CoordinateTransformation(inSpatialRef, outSpatialRef)

# get the input layer
inDataSet = driver.Open(r'/home/user/jupyter/notebook_gallery/' + name + ".geojson")
inLayer = inDataSet.GetLayer()

# create the output layer
outputGeoJSON = r'/home/user/jupyter/notebook_gallery/' + name + "_wgs" + ".geojson"
if os.path.exists(outputGeoJSON):
    driver.DeleteDataSource(outputGeoJSON)
outDataSet = driver.CreateDataSource(outputGeoJSON)
outLayer = outDataSet.CreateLayer(name + "_wgs", geom_type=ogr.wkbMultiPolygon)

# add fields
inLayerDefn = inLayer.GetLayerDefn()
for i in range(0, inLayerDefn.GetFieldCount()):
    fieldDefn = inLayerDefn.GetFieldDefn(i)
    outLayer.CreateField(fieldDefn)

# get the output layer's feature definition
outLayerDefn = outLayer.GetLayerDefn()

# loop through the input features
inFeature = inLayer.GetNextFeature()
while inFeature:
    # get the input geometry
    geom = inFeature.GetGeometryRef()
    # reproject the geometry
    geom.Transform(coordTrans)
    # create a new feature
    outFeature = ogr.Feature(outLayerDefn)
    # set the geometry and attribute
    outFeature.SetGeometry(geom)
    for i in range(0, outLayerDefn.GetFieldCount()):
        outFeature.SetField(outLayerDefn.GetFieldDefn(i).GetNameRef(), inFeature.GetField(i))
    # add the feature to the shapefile
    outLayer.CreateFeature(outFeature)
    # dereference the features and get the next input feature
    outFeature = None
    inFeature = inLayer.GetNextFeature()

# Save and close the shapefiles
inDataSet = None
outDataSet = None

# -------- CARREGAR WGS.GEOJSON PARA O CÓDIGO


poligonos = os.path.join(name + "_wgs" + ".geojson")



# -------- APLICAR ESTILO DE COR PARA CADA VALOR
    
   
map_data = pd.DataFrame({
    'DN':[1, 2, 3],
    'value':[1, 2, 3]
})


map_dict = map_data.set_index('DN')['value'].to_dict()

color_scale = LinearColormap(['yellow','red'], vmin = 1, vmax = 3)

def get_color(feature):
    value = map_dict.get(feature['properties']['DN'])
    if value is 1:
        return '#8c8c8c' # MISSING -> gray
    elif value is 2:
        return 'red'
    elif value is 3:
        return '#8c8c8c'
#     else:
#         return color_scale(value)

print(map_dict)

m.add_child(color_scale)   
    
# -------- PLOTAR O WGS GEOJSON NO MAPA
    
    
folium.GeoJson(
    data = '/home/user/jupyter/notebook_gallery/' + name + "_wgs" + ".geojson",
    name = 'Mapa de Proveniencia da ' + name,
    style_function = lambda feature: {
        'fillColor': get_color(feature),
        'fillOpacity': 0.7,
        'color' : 'black',
        'weight' : 1,
    }    
).add_to(m)


# -------- ADICIONAR CONTROLE DE CAMADAS 


folium.LayerControl().add_to(m)


# -------- SALVAR MAPA EM HTML


m.save('mapa.html')

m

Nome da amostra (sem espaco): sample_2233
Susceptibilidade magnetica (de 0.00 a 1.00): 0.5



10420
{1: 1, 2: 2, 3: 3}


NameError: name 'fdgsdg' is not defined